**Analyse des logs**

Fatima Zahra Chalf

In [1]:
!pip install pyspark


In [2]:
from pyspark import SparkContext


try:
    sc.stop()
except:
    pass

sc = SparkContext(appName='LogCountPerMinute')

In [5]:

file_path = "/content/Android.log"

try:
    log_RDD = sc.textFile(file_path)
except Exception as e:
    print(f"Error loading file: {e}")

In [7]:

def extract_minute_key(line):
    try:
        parts = line.strip().split()


        if len(parts) < 2:
            return None

        date_part = parts[0]
        time_part = parts[1]

        # On vérifie que c'est bien un format d'heure (contient ':')
        if ':' not in time_part:
            return None

        # On coupe les secondes pour garder HH:mm
        minute_part = time_part[:5]

        # Clé finale : "12-17 19:31"
        return f"{date_part} {minute_part}"
    except:
        return None

In [9]:
# 6. Exécution du Job Spark (Map -> Reduce -> Sort)
counts = (
    log_RDD
    .map(lambda line: (extract_minute_key(line), 1))
    .filter(lambda x: x[0] is not None)
    .reduceByKey(lambda a, b: a + b)
    .sortByKey()
)

In [11]:

try:
    results = counts.collect()

    print(f"\n--- Résultats pour {file_path} ---")
    print(f"Nombre de créneaux (minutes) trouvés : {len(results)}\n")

    for timestamp, count in results:
        print(f"Minute : {timestamp} -> {count} messages")

except Exception as e:
    print(f"Une erreur est survenue. Vérifie que le fichier est bien présent ici : {file_path}")
    print(f"Détail de l'erreur : {e}")


--- Résultats pour /content/Android.log ---
Nombre de créneaux (minutes) trouvés : 139

Minute : 12-17 19:31 -> 11782 messages
Minute : 12-17 19:32 -> 6021 messages
Minute : 12-17 19:33 -> 9429 messages
Minute : 12-17 19:34 -> 3799 messages
Minute : 12-17 19:35 -> 5580 messages
Minute : 12-17 19:36 -> 955 messages
Minute : 12-17 19:37 -> 204 messages
Minute : 12-17 19:40 -> 3769 messages
Minute : 12-17 19:41 -> 93 messages
Minute : 12-17 19:42 -> 2207 messages
Minute : 12-17 19:43 -> 164 messages
Minute : 12-17 19:44 -> 188 messages
Minute : 12-17 19:45 -> 441 messages
Minute : 12-17 19:47 -> 538 messages
Minute : 12-17 19:48 -> 363 messages
Minute : 12-17 19:50 -> 209 messages
Minute : 12-17 19:51 -> 150 messages
Minute : 12-17 19:52 -> 311 messages
Minute : 12-17 19:54 -> 343 messages
Minute : 12-17 19:55 -> 3628 messages
Minute : 12-17 19:56 -> 6351 messages
Minute : 12-17 19:57 -> 8750 messages
Minute : 12-17 19:58 -> 7329 messages
Minute : 12-17 19:59 -> 4455 messages
Minute : 12

In [12]:
#  Analyse par Heure et par Niveau


def extract_hour_and_level(line):
    try:
        parts = line.strip().split()


        if len(parts) < 5:
            return None

        date_part = parts[0]      # Ex: 12-17
        time_part = parts[1]      # Ex: 19:31:36.263
        level = parts[4]          # Ex: I, D, E...

        # On garde seulement l'heure (les 2 premiers chiffres)
        hour = time_part[:2]


        return ((f"{date_part} {hour}h", level), 1)
    except:
        return None

# 2. MapReduce
level_counts = (
    log_RDD
    .map(extract_hour_and_level)
    .filter(lambda x: x is not None)
    .reduceByKey(lambda a, b: a + b)
    .sortByKey()
)

# 3. Affichage
results_levels = level_counts.collect()

print("\n--- Répartition des messages par Heure et par Niveau ---")
# Petit dictionnaire pour traduire les lettres
level_names = {'I': 'INFO', 'D': 'DEBUG', 'V': 'VERBOSE', 'W': 'WARNING', 'E': 'ERROR'}

for (key, count) in results_levels:
    date_hour, level_code = key
    level_full = level_names.get(level_code, level_code)
    print(f"[{date_hour}] {level_full} ({level_code}) : {count}")


--- Répartition des messages par Heure et par Niveau ---
[12-17 19h] DEBUG (D) : 32534
[12-17 19h] ERROR (E) : 9474
[12-17 19h] INFO (I) : 29725
[12-17 19h] VERBOSE (V) : 2046
[12-17 19h] WARNING (W) : 3280
[12-17 20h] DEBUG (D) : 42570
[12-17 20h] ERROR (E) : 10727
[12-17 20h] INFO (I) : 72732
[12-17 20h] VERBOSE (V) : 5479
[12-17 20h] WARNING (W) : 6981
[12-17 21h] DEBUG (D) : 52940
[12-17 21h] ERROR (E) : 14916
[12-17 21h] INFO (I) : 103542
[12-17 21h] VERBOSE (V) : 6767
[12-17 21h] WARNING (W) : 9430
